# Setup Feature Tracking Tables

**Purpose**: One-time setup to create Delta tables for Feature Release Tracking

**Tables Created**:
1. `feature_releases` - Microsoft Fabric feature releases from What's New
2. `preview_features_active` - Preview features currently activated in tenant
3. `feature_alerts` - Alerts for preview feature activations

**Run Once**: This notebook only needs to be executed once during initial setup

**Prerequisites**: 
- FUAM Lakehouse must exist
- `tenant_settings` table should already be created by FUAM

In [ ]:
from pyspark.sql.types import *
from pyspark.sql import functions as F
from datetime import datetime

## Step 1: Create `feature_releases` Table

In [ ]:
print("🔄 Creating table: feature_releases")
print("=" * 60)

schema_feature_releases = StructType([
    StructField("feature_id", StringType(), False),
    StructField("feature_name", StringType(), False),
    StructField("workload", StringType(), True),
    StructField("release_date", TimestampType(), False),
    StructField("status", StringType(), True),
    StructField("is_preview", BooleanType(), False),
    StructField("source_url", StringType(), True),
    StructField("extracted_date", TimestampType(), False)
])

# Create empty DataFrame
df_feature_releases = spark.createDataFrame([], schema_feature_releases)

# Write to Delta
table_path = "Tables/feature_releases"
df_feature_releases.write.format("delta").mode("overwrite").save(table_path)

print("✅ Table created: feature_releases")
print(f"   Location: {table_path}")
print("\n   Schema:")
for field in schema_feature_releases.fields:
    print(f"     - {field.name}: {field.dataType.simpleString()}")

## Step 2: Create `preview_features_active` Table

In [ ]:
print("\n🔄 Creating table: preview_features_active")
print("=" * 60)

schema_preview_active = StructType([
    StructField("setting_name", StringType(), False),
    StructField("feature_id", StringType(), False),
    StructField("feature_name", StringType(), False),
    StructField("workload", StringType(), True),
    StructField("similarity_score", DoubleType(), False),
    StructField("is_enabled", BooleanType(), False),
    StructField("delegate_to_tenant", BooleanType(), True),
    StructField("detected_date", TimestampType(), False),
    StructField("release_date", TimestampType(), True),
    StructField("status", StringType(), True),
    StructField("source_url", StringType(), True),
    StructField("days_since_release", IntegerType(), True)
])

# Create empty DataFrame
df_preview_active = spark.createDataFrame([], schema_preview_active)

# Write to Delta
table_path = "Tables/preview_features_active"
df_preview_active.write.format("delta").mode("overwrite").save(table_path)

print("✅ Table created: preview_features_active")
print(f"   Location: {table_path}")
print("\n   Schema:")
for field in schema_preview_active.fields:
    print(f"     - {field.name}: {field.dataType.simpleString()}")

## Step 3: Create `feature_alerts` Table

In [ ]:
print("\n🔄 Creating table: feature_alerts")
print("=" * 60)

schema_alerts = StructType([
    StructField("alert_id", StringType(), False),
    StructField("feature_id", StringType(), False),
    StructField("feature_name", StringType(), False),
    StructField("workload", StringType(), True),
    StructField("alert_type", StringType(), False),
    StructField("severity", StringType(), False),
    StructField("message", StringType(), False),
    StructField("days_since_release", IntegerType(), True),
    StructField("similarity_score", DoubleType(), True),
    StructField("alert_date", TimestampType(), False),
    StructField("acknowledged", BooleanType(), False)
])

# Create empty DataFrame
df_alerts = spark.createDataFrame([], schema_alerts)

# Write to Delta
table_path = "Tables/feature_alerts"
df_alerts.write.format("delta").mode("overwrite").save(table_path)

print("✅ Table created: feature_alerts")
print(f"   Location: {table_path}")
print("\n   Schema:")
for field in schema_alerts.fields:
    print(f"     - {field.name}: {field.dataType.simpleString()}")

## Step 4: Verify Tables Created

In [ ]:
print("\n📊 Verification Summary")
print("=" * 60)

tables_to_verify = [
    "feature_releases",
    "preview_features_active", 
    "feature_alerts"
]

all_tables_exist = True

for table_name in tables_to_verify:
    table_path = f"Tables/{table_name}"
    try:
        from delta.tables import DeltaTable
        
        if DeltaTable.isDeltaTable(spark, table_path):
            df = spark.read.format("delta").load(table_path)
            count = df.count()
            print(f"✅ {table_name}: EXISTS (rows={count})")
        else:
            print(f"❌ {table_name}: NOT A DELTA TABLE")
            all_tables_exist = False
    except Exception as e:
        print(f"❌ {table_name}: ERROR - {e}")
        all_tables_exist = False

print("=" * 60)

if all_tables_exist:
    print("\n🎉 SUCCESS! All tables created successfully")
    print("\nNext Steps:")
    print("1. Run '01_Transfer_Feature_Releases_Unit' to populate feature_releases")
    print("2. Run '02_Transfer_Preview_Features_Unit' to detect activated previews")
    print("3. Run '03_Transfer_Feature_Alerts_Unit' to generate alerts")
    print("\nOR")
    print("→ Run 'Load_Feature_Tracking_E2E' pipeline to execute all steps")
else:
    print("\n⚠️ WARNING: Some tables failed to create. Review errors above.")

## Step 5: Create Sample Queries View (Optional)

In [ ]:
print("\n🔄 Creating helper views for SQL Endpoint...")

# View 1: Active Preview Features (for quick querying)
spark.sql("""
    CREATE OR REPLACE VIEW vw_active_preview_features AS
    SELECT 
        feature_name,
        workload,
        setting_name,
        days_since_release,
        similarity_score,
        release_date,
        detected_date
    FROM preview_features_active
    WHERE is_enabled = true
    ORDER BY detected_date DESC
""")
print("✅ Created view: vw_active_preview_features")

# View 2: Unacknowledged Critical Alerts
spark.sql("""
    CREATE OR REPLACE VIEW vw_critical_alerts AS
    SELECT 
        alert_id,
        feature_name,
        workload,
        alert_type,
        severity,
        message,
        alert_date
    FROM feature_alerts
    WHERE acknowledged = false 
      AND severity IN ('Critical', 'Warning')
    ORDER BY alert_date DESC
""")
print("✅ Created view: vw_critical_alerts")

# View 3: Feature Release Timeline
spark.sql("""
    CREATE OR REPLACE VIEW vw_feature_timeline AS
    SELECT 
        feature_name,
        workload,
        status,
        is_preview,
        release_date,
        DATEDIFF(CURRENT_DATE(), release_date) as days_since_release
    FROM feature_releases
    ORDER BY release_date DESC
""")
print("✅ Created view: vw_feature_timeline")

print("\n✅ All views created successfully")
print("   → These views are accessible via SQL Endpoint")

## ✅ Setup Complete!

In [ ]:
print("\n" + "=" * 60)
print("🎉 FEATURE TRACKING SETUP COMPLETED!")
print("=" * 60)

print("\n📋 Tables Created:")
print("   ✅ feature_releases")
print("   ✅ preview_features_active")
print("   ✅ feature_alerts")

print("\n📋 Views Created:")
print("   ✅ vw_active_preview_features")
print("   ✅ vw_critical_alerts")
print("   ✅ vw_feature_timeline")

print("\n🚀 Ready to Run:")
print("   → Execute 'Load_Feature_Tracking_E2E' pipeline")
print("   → Or run individual Unit notebooks")

print("\n📊 Query Examples:")
print("   SELECT * FROM vw_active_preview_features;")
print("   SELECT * FROM vw_critical_alerts;")
print("   SELECT * FROM vw_feature_timeline WHERE is_preview = true;")

print("\n" + "=" * 60)